# Metody Obliczeniowe w Nauce i Technice
# Laboratorium 4: Symulowane wyżarzanie
## Przemysław Roman

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

def time_exec(func, *args):
    start_time = time.time()
    result = func(*args)
    return result, time.time() - start_time

## Zadanie 1 TSP

Wygeneruj chmurę n losowych punktów w 2D, a następnie zastosuj algorytm symulowa-
nego wyżarzania do przybliżonego rozwiązania problemu komiwojażera dla tych punk-
tów.
1. Przedstaw wizualizację otrzymanego rozwiązania dla 3 różnych wartości n oraz 3
różnych układów punktów w 2D (rozkład jednostajny, rozkład normalny z czterema
różnymi grupami parametrów, dziewięć odseparowanych grup punktów).
2. Zbadaj wpływ sposobu generacji sąsiedniego stanu (consecutive swap vs. arbitrary
swap) oraz funkcji zmiany temperatury na zbieżność procesu optymalizacji.
3. Przedstaw wizualizację (saoptimset) działania procedury minimalizującej funkcję
celu.

In [51]:
def generate_points(n, min_x, max_x, min_y, max_y):
    return np.array([(np.random.randint(min_x, max_x), np.random.randint(min_y, max_y)) for _ in range(n)])

def euclidian_distance(A, B):
    return np.sqrt((A[0]-B[0])**2 + (A[1]-B[1])**2)

def calculate_cost(x, *args):
    arbitrary_swap(points, int(x[0]), int(x[1]))
    dist = 0
    n = len(points)
    for i in range(n):
        dist += euclidian_distance(points[i], points[(i+1)%n])
        # np.linalg.norm(points[x[i]]-points[x[(i+1)%n]])
    return dist

def arbitrary_swap(order, i, j):
    n = len(order)
    rng = np.random.default_rng()
    # i, j = rng.choice(n, size=2, replace=False)
    order[i], order[j] = order[j], order[i]

from scipy import optimize

n = 100
bounds = [[0, 100], [0, 100]]
points = generate_points(n, 0, 10, -5, 5)
order = [i for i in range(n)]
best_order = points
# for i in range(10):
    # print(calculate_cost(points))
print(optimize.dual_annealing(calculate_cost, bounds))

     fun: 0.0
 message: ['Maximum number of iteration reached']
    nfev: 4184
    nhev: 0
     nit: 1000
    njev: 61
  status: 0
 success: True
       x: array([35.30054041, 46.84118009])
